## 任务1：模型训练与预测

- 步骤1：导入LightGBM库
- 步骤2：使用LGBMClassifier对iris进行训练。
- 步骤3：将预测的模型对iris进行预测

In [35]:
import lightgbm as lgb
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score,accuracy_score,auc
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [24]:
iris = datasets.load_iris()
X = pd.DataFrame(data = iris.data , columns=iris.feature_names)
y = pd.DataFrame(iris.target,columns=['target'])

data = pd.concat([X,y],axis=1)
data.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [15]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=2022)


In [39]:
lgb_model = lgb.LGBMClassifier(objective= 'multiclass',
                               metric = 'multi_logloss',
                               boosting_type= 'gbdt',
                               n_estimators= 100,
                               learning_rate= 0.2,
                               n_jobs= -1,
                               random_state= 2022
                               )


lgb_model.fit(X_train,y_train)
y_pre = lgb_model.predict(X_test)
print('预测值：{}'.format(y_pre))
print('模型f1值：{}'.format(f1_score(y_test,y_pre,average='macro')))

预测值：[2 2 0 2 0 0 1 1 0 1 1 2 1 2 2 0 1 1 2 1 0 0 2 0 2 2 2 0 1 2]
模型f1值：0.9670588235294119


## 任务2：模型保存与加载

https://github.com/microsoft/LightGBM/blob/master/examples/python-guide/advanced_example.py
- 步骤1：将任务1训练得到的模型，使用pickle进行保存。
- 步骤2：将任务1训练得到的模型，使用json进行保存。
- 步骤3：加载步骤1和步骤2的模型，并进行预测。